# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [1]:
import pandas as pd
import numpy as np


# Задание 1 -  Базовое изучение
### Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.
 
Из сопроводительного письма "names" видно, что столбцы в данных не подписаны, а имеют нумерацию и описательную характеристику в записке. Вместо NaN использован "?", проверено isnan без параметра "na_values = ?"  
при простой загрузке первый ряд данных становится именем столбца, добавим параметр headers = none при загрузке CSV 
и переименуем столбцы для удобства. 

сразу отбрасываемые столбцы: Age - имеет 2 варианта, меньше 6 месяцев и старше. Малоинформативен с т.з. длительности жизни лошади *Hospital Number* - Если не рассматривать гипотезу о коновалах (смертность в отдельных госпиталях), - малоинформативен. lesion site 1-2-3 - столбцы интересны тем, что NA выражены в них "0", также можно было бы проверить гипотезу про связь между наличием множественных лезий и смертностью, но лимит на 8 столбцов.

выбираем столбцы: 
1) surgery? - была ли у лошади операция? Низкий процент потерь данных. Категорийный

2) *rectal temperature* - зависит состояние лошади, есть ли воспаление. Возможно есть связь между гибелью и температурой, т.к. перед гибелью температура падает. Числовой. Выше 37.8 - не норма. 

3) *temperature of extremities* - температура конечностей. значения 3-4 могут обозначать шок. Категорийный. Возможна связь со смертностью. 

4) *pulse* - высокий пульс может быть признаком воспаления. Числовой. Норма - 25-40. 

5) *total protein* - есть ли связь между мышечным истощением и гибелью лошади? числовой. Норма 6-7.5 

6) *abdominal distension* - вздутие брюшной полости. Может быть признаком протекающего заболевания. Заявлен как важный параметр. Категорийный 

7) *surgical lesion* - лезии при хирургическом вмешательстве. Может быть признаком осложнения хирургии. Категорийный 

8) *outcome* - исход: погибла, выжила или усыплена лошадь. Вполне возможно, что есть связь между исходом и оперативным вмешательством. Категорийный

Основные показатели и описание - далее



In [2]:
column_names = {0:  "surgery?",
1:  "Age",
2:  "Hospital Number",
3:  'rectal temperature',
4:  "pulse",
5:  "respiratory rate",
6:  "temperature of extremities",
7:  "peripheral pulse",
8:  "mucous membranes",
9: "capillary refill time",
10: "pain",
11: "peristalsis",
12: "abdominal distension",
13: "nasogastric tube",
14: "nasogastric reflux",
15: "nasogastric reflux PH",
16: "rectal examination - feces",
17: "abdomen",
18: "packed cell volume",
19: "total protein",
20: "abdominocentesis appearance",
21: "abdomcentesis total protein",
22: "outcome",
23: "surgical lesion?",
24: "lesion site 1",
25: "lesion site 2",
26: "lesion site 3",
27: "cp_data",
               }
log = pd.read_csv("horse_data.csv", header=None, na_values = "?").rename(columns=column_names)
log


,surgery?,Age,Hospital Number,rectal temperature,pulse,respiratory rate,temperature of extremities,peripheral pulse,mucous membranes,capillary refill time,...,packed cell volume,total protein,abdominocentesis appearance,abdomcentesis total protein,outcome,surgical lesion?,lesion site 1,lesion site 2,lesion site 3,cp_data
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0,NaN,2.0,...,45.0,8.4,NaN,NaN,2.0,2,11300,0,0,2
1,1.0,1,534817,39.2,88.0,20.0,NaN,NaN,4.0,1.0,...,50.0,85.0,2.0,2.0,3.0,2,2208,0,0,2
2,2.0,1,530334,38.3,40.0,24.0,1.0,1.0,3.0,1.0,...,33.0,6.7,NaN,NaN,1.0,2,0,0,0,1
3,1.0,9,5290409,39.1,164.0,84.0,4.0,1.0,6.0,2.0,...,48.0,7.2,3.0,5.3,2.0,1,2208,0,0,1
4,2.0,1,530255,37.3,104.0,35.0,NaN,NaN,6.0,2.0,...,74.0,7.4,NaN,NaN,2.0,2,4300,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1.0,1,533886,NaN,120.0,70.0,4.0,NaN,4.0,2.0,...,55.0,65.0,NaN,NaN,3.0,2,3205,0,0,2
296,2.0,1,527702,37.2,72.0,24.0,3.0,2.0,4.0,2.0,...,44.0,NaN,3.0,3.3,3.0,1,2208,0,0,1
297,1.0,1,529386,37.5,72.0,30.0,4.0,3.0,4.0,1.0,...,60.0,6.8,NaN,NaN,2.0,1,3205,0,0,2
298,1.0,1,530612,36.5,100.0,24.0,3.0,3.0,3.0,1.0,...,50.0,6.0,3.0,3.4,1.0,1,2208,0,0,1


In [3]:
log.isna().mean()*100

surgery?                        0.333333
Age                             0.000000
Hospital Number                 0.000000
rectal temperature             20.000000
pulse                           8.000000
respiratory rate               19.333333
temperature of extremities     18.666667
peripheral pulse               23.000000
mucous membranes               15.666667
capillary refill time          10.666667
pain                           18.333333
peristalsis                    14.666667
abdominal distension           18.666667
nasogastric tube               34.666667
nasogastric reflux             35.333333
nasogastric reflux PH          82.333333
rectal examination - feces     34.000000
abdomen                        39.333333
packed cell volume              9.666667
total protein                  11.000000
abdominocentesis appearance    55.000000
abdomcentesis total protein    66.000000
outcome                         0.333333
surgical lesion?                0.000000
lesion site 1   

In [4]:
small_log = log[["surgery?","rectal temperature", "temperature of extremities", "pulse","total protein",
                "abdominal distension", "surgical lesion?", "outcome"
                ]]
small_log[["rectal temperature",'pulse','total protein',]].describe()


,rectal temperature,pulse,total protein
count,240.000000,276.000000,267.000000
mean,38.167917,71.913043,24.456929
std,0.732289,28.630557,27.475009
min,35.400000,30.000000,3.300000
25%,37.800000,48.000000,6.500000
50%,38.200000,64.000000,7.500000
75%,38.500000,88.000000,57.000000
max,40.800000,184.000000,89.000000


# Задание 1, Задание 2. Работа с выбросами
1) rectal temperature: 
на первый взгляд выбросы отсутствуют, т.к. минимальные и максимальные значения в пределах "живой" нормы животного. Низкий уровень стандартного отклонения говорит о низком разбросе значений множества от средней величины. Средняя и медианная величины отличаются незначительно. при обнаружении корреляции с другими параметрами можно будет достроить недостающие данные на основе расчетных величин.

2) pulse:
максимальные значения находятся в пределах "живой" нормы лошади и могут быть обусловлены наличием лезий / болевым синдромом. В 8-мистолбовой выборке данных болевой синдром и наличие нехирургических лезий отсутствует. можно будет предположить, что если исход смерть / эвтаназия, то данные не являются выбросами, а результатом септического шока. Среднее и медианные значения - повышенные, могут быть свидетельством стресса лошади. 

3) total protein
третий квартиль говорит о том, что имеется большой процент данных, выходящих за пределы нормы почти в 10 раз. Имеется фактическая зависимость между уровнем packed cell volume (на основе вет.литературы). Понять, являются ли максимальные и минимальные величины выбросами на основании ошибки мне не представляется возможным. 
# Задание 3. Работа с пропусками
Поскольку в учебных целях мы пока будем предполагать, что наши пропуски всегда полностью случайны.
будут произведены следующие допущения:
1) удалены строки, а которых больше, чем 1 пропуск.

2) тип данных в категориальных столбцах будет изменен на Int, целочисленный. 

3) замены категориальных пропусков будут произведены модами

4) замены числовых пропусков будут произведены медианами

In [5]:
filled_small_log = small_log.dropna(thresh=7).copy()
print(filled_small_log[["temperature of extremities","surgery?","abdominal distension"]].mode())
filled_small_log['surgery?'] = filled_small_log['surgery?'].fillna(filled_small_log['surgery?'].mode(dropna=True))
filled_small_log['rectal temperature'] = filled_small_log['rectal temperature'].fillna(filled_small_log['rectal temperature'].median())
filled_small_log['temperature of extremities'] = filled_small_log['temperature of extremities'].fillna(3)
filled_small_log['pulse'] = filled_small_log['pulse'].fillna(filled_small_log['pulse'].median())
filled_small_log['total protein'] = filled_small_log['total protein'].fillna(filled_small_log['total protein'].median())
filled_small_log['abdominal distension'] = filled_small_log['abdominal distension'].fillna(1)
filled_small_log['surgical lesion?'] = filled_small_log['surgical lesion?'].fillna(filled_small_log['surgical lesion?'].mode())
filled_small_log.info()

   temperature of extremities  surgery?  abdominal distension
0                         3.0       1.0                   1.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 238 entries, 0 to 298
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    238 non-null    float64
 1   rectal temperature          238 non-null    float64
 2   temperature of extremities  238 non-null    float64
 3   pulse                       238 non-null    float64
 4   total protein               238 non-null    float64
 5   abdominal distension        238 non-null    float64
 6   surgical lesion?            238 non-null    int64  
 7   outcome                     238 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 16.7 KB


## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.